# News Headline Sentiment Classification

End-to-end workflow that turns raw financial news headlines for large-cap tickers into Bullish / Neutral / Bearish sentiment labels, compares linear models, and surfaces the most influential tokens.


## Introduction

- Fetch fresh headlines for multiple tickers via NewsAPI
- Manually label ~250 rows (`data/labeled_headlines.csv`) as Bullish / Bearish / Neutral
- Train Logistic Regression and Linear SVM models on TF-IDF text + ticker one-hot features
- Evaluate using accuracy + F1, visualize confusion, extract top driver tokens, and package artifacts for sharing


> 📌 **Manual labeling requirement**
>
> 1. Run Step 1 to generate `data/raw_headlines.csv`.
> 2. Open that file, add a `sentiment` column with values in {Bullish, Neutral, Bearish} for roughly 250 rows.
> 3. Save it as `data/labeled_headlines.csv`, then continue with the notebook.


## Setup


In [ ]:
import os
import sys
from pathlib import Path

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from dotenv import load_dotenv
from scipy import sparse
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# Resolve project root so we can import from src/
PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / "src").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
sys.path.append(str(PROJECT_ROOT))

load_dotenv(PROJECT_ROOT / ".env")

from src.config import (  # noqa: E402
    CLASS_LABELS,
    FIGURES_DIR,
    LABELED_DATA_PATH,
    MODEL_ARTIFACT_PATHS,
    MODEL_COMPARISON_CSV,
    MODEL_COMPARISON_MD,
    RAW_DATA_PATH,
    REPORTS_DIR,
    TICKERS,
)
from src.data_fetch import collect_all_headlines  # noqa: E402
from src.explain import (  # noqa: E402
    get_feature_names,
    get_top_tokens_per_class,
    save_top_tokens_csv,
    save_top_tokens_plot,
)
from src.preprocess import (  # noqa: E402
    build_vectorizers,
    clean_text,
    load_labeled_data,
    transform_features,
)
from src.train import (  # noqa: E402
    evaluate_model,
    train_linear_svc,
    train_logistic_regression,
)

pd.set_option("display.max_colwidth", 120)
sns.set_theme(style="whitegrid", context="talk")
FIGURES_DIR.mkdir(parents=True, exist_ok=True)
REPORTS_DIR.mkdir(parents=True, exist_ok=True)



In [ ]:
print("Configured tickers:", TICKERS)
print("Raw data path:", RAW_DATA_PATH)
print("Labeled data path:", LABELED_DATA_PATH)
print("Figures dir:", FIGURES_DIR)
print("Reports dir:", REPORTS_DIR)



## Step 1 – Fetch raw headlines (one-time)


Run the cell below whenever you need a fresh batch of unlabeled headlines. It attempts NewsAPI first, then automatically falls back to the manual scraper (Google News + Yahoo Finance) if the API key is missing, invalid, rate-limited, or if you force manual mode. Either way, the output is `data/raw_headlines.csv` with the same schema. Skip if you already have a labeled dataset.


> 💡 To force manual scraping (skip the API entirely), set the environment variable before running the cell:
>
> ```python
> import os
> os.environ["USE_MANUAL_SCRAPE"] = "1"
> ```
>
> Leave it unset to let the code try NewsAPI first and fall back automatically on failure.


In [ ]:
raw_df = None
try:
    raw_df = collect_all_headlines()
except Exception as exc:
    print(f"[notebook] Automatic collection failed: {exc}")
    if RAW_DATA_PATH.exists():
        raw_df = pd.read_csv(RAW_DATA_PATH)
        print(f"Loaded cached raw headlines from {RAW_DATA_PATH}")

if raw_df is not None:
    display(raw_df.head())
    print(f"Total raw headlines: {len(raw_df):,}")
    print(raw_df['ticker'].value_counts())
else:
    print(
        "No raw headlines available yet. Provide NEWS_API_KEY or enable manual scraping and rerun."
    )



> ✋ **Pause here:** Open `data/raw_headlines.csv`, label ~250 rows with `sentiment` in {Bullish, Neutral, Bearish}, and save as `data/labeled_headlines.csv`. Continue once labeling is complete.


## Step 2 – Load labeled data


In [ ]:
if not LABELED_DATA_PATH.exists():
    raise FileNotFoundError(
        f"Labeled dataset not found at {LABELED_DATA_PATH}. Create it before proceeding."
    )

labeled_df = load_labeled_data()
print(f"Labeled rows: {len(labeled_df):,}")
display(labeled_df.head())

class_counts = labeled_df["sentiment"].value_counts().reindex(CLASS_LABELS, fill_value=0)
ax = class_counts.plot(kind="bar", color=["#1b7837", "#f0ad4e", "#d73027"], figsize=(6, 4))
ax.set_ylabel("Count")
ax.set_title("Class distribution")
plt.show()

min_ratio = class_counts.min() / class_counts.sum()
if min_ratio < 0.15:
    print("⚠️  Consider labeling more data for the minority class to balance training.")



## Step 3 – Preprocess + feature engineering


In [ ]:
tfidf_vectorizer, ticker_encoder = build_vectorizers(labeled_df)
X, y = transform_features(labeled_df, tfidf_vectorizer, ticker_encoder)
print(f"Feature matrix shape: {X.shape}")
print(f"Label distribution: {dict(zip(*np.unique(y, return_counts=True)))}")



## Step 4 – Train/validation split


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
train_counts = dict(zip(*np.unique(y_train, return_counts=True)))
test_counts = dict(zip(*np.unique(y_test, return_counts=True)))
print("Train label counts:", train_counts)
print("Test label counts:", test_counts)



## Step 5 – Train & evaluate models


In [ ]:
logreg_model = train_logistic_regression(X_train, y_train)
logreg_metrics = evaluate_model(
    logreg_model,
    X_train,
    y_train,
    X_test,
    y_test,
    model_name="Logistic Regression",
    figures_dir=FIGURES_DIR,
)
logreg_metrics



In [ ]:
linearsvc_model = train_linear_svc(X_train, y_train)
linearsvc_metrics = evaluate_model(
    linearsvc_model,
    X_train,
    y_train,
    X_test,
    y_test,
    model_name="Linear SVC",
    figures_dir=FIGURES_DIR,
)
linearsvc_metrics



## Step 6 – Explainability (top tokens)


In [ ]:
feature_names = get_feature_names(tfidf_vectorizer, ticker_encoder)

logreg_top_tokens = get_top_tokens_per_class(logreg_model, feature_names, top_k=15)
logreg_plot_path = save_top_tokens_plot(
    logreg_top_tokens,
    model_name="Logistic Regression",
    figures_dir=FIGURES_DIR,
)
logreg_csv_path = save_top_tokens_csv(
    logreg_top_tokens,
    model_name="Logistic Regression",
    reports_dir=REPORTS_DIR,
)
print("Logistic Regression explainability saved to:")
print(" •", logreg_plot_path)
print(" •", logreg_csv_path)

pd.DataFrame(logreg_top_tokens["Bullish"], columns=["feature", "weight"]).head()



In [ ]:
linearsvc_top_tokens = get_top_tokens_per_class(linearsvc_model, feature_names, top_k=15)
linearsvc_plot_path = save_top_tokens_plot(
    linearsvc_top_tokens,
    model_name="Linear SVC",
    figures_dir=FIGURES_DIR,
)
linearsvc_csv_path = save_top_tokens_csv(
    linearsvc_top_tokens,
    model_name="Linear SVC",
    reports_dir=REPORTS_DIR,
)
print("Linear SVC explainability saved to:")
print(" •", linearsvc_plot_path)
print(" •", linearsvc_csv_path)

pd.DataFrame(linearsvc_top_tokens["Bearish"], columns=["feature", "weight"]).head()



In [ ]:
def tokens_dict_to_frame(tokens_dict, model_name):
    rows = []
    for label, tuples in tokens_dict.items():
        for feature, weight in tuples:
            rows.append({"model": model_name, "class": label, "feature": feature, "weight": weight})
    return pd.DataFrame(rows)

explain_df = pd.concat(
    [
        tokens_dict_to_frame(logreg_top_tokens, "Logistic Regression"),
        tokens_dict_to_frame(linearsvc_top_tokens, "Linear SVC"),
    ],
    ignore_index=True,
)
explain_df.head(15)



## Step 7 – Compare models


In [ ]:
metrics_df = pd.DataFrame([logreg_metrics, linearsvc_metrics])
metrics_df

metrics_df.to_csv(MODEL_COMPARISON_CSV, index=False)

logreg_cm = confusion_matrix(y_test, logreg_model.predict(X_test), labels=CLASS_LABELS)
linearsvc_cm = confusion_matrix(y_test, linearsvc_model.predict(X_test), labels=CLASS_LABELS)


def describe_confusion(cm):
    cm_copy = cm.copy()
    np.fill_diagonal(cm_copy, 0)
    max_idx = np.unravel_index(np.argmax(cm_copy), cm_copy.shape)
    return CLASS_LABELS[max_idx[0]], CLASS_LABELS[max_idx[1]], int(cm_copy[max_idx])

logreg_conf = describe_confusion(logreg_cm)
linearsvc_conf = describe_confusion(linearsvc_cm)

best_macro = metrics_df.loc[metrics_df["macro_f1"].idxmax()]
best_weighted = metrics_df.loc[metrics_df["weighted_f1"].idxmax()]

summary_lines = [
    f"- Macro F1 leader: {best_macro['model_name']} ({best_macro['macro_f1']:.3f}).",
    f"- Weighted F1 leader: {best_weighted['model_name']} ({best_weighted['weighted_f1']:.3f}).",
    f"- Largest confusion (LogReg): {logreg_conf[0]} → {logreg_conf[1]} ({logreg_conf[2]} cases).",
    f"- Largest confusion (Linear SVC): {linearsvc_conf[0]} → {linearsvc_conf[1]} ({linearsvc_conf[2]} cases).",
    "- Both models remain interpretable thanks to linear weights over TF-IDF + ticker features.",
]

markdown_table = metrics_df.to_markdown(index=False, floatfmt=".3f")
REPORTS_DIR.mkdir(parents=True, exist_ok=True)
with open(MODEL_COMPARISON_MD, "w", encoding="utf-8") as f:
    f.write("# Model Comparison\n\n")
    f.write(markdown_table + "\n\n")
    f.write("## Highlights\n")
    for line in summary_lines:
        f.write(line + "\n")

print("Saved model comparison to:")
print(" •", MODEL_COMPARISON_CSV)
print(" •", MODEL_COMPARISON_MD)



## Step 8 – Inference helper


In [ ]:
model_registry = {
    "Logistic Regression": {"model": logreg_model, "metrics": logreg_metrics},
    "Linear SVC": {"model": linearsvc_model, "metrics": linearsvc_metrics},
}
best_model_name = max(
    model_registry.keys(),
    key=lambda name: model_registry[name]["metrics"]["macro_f1"],
)
print(f"Using {best_model_name} for inference by default.")


def predict_sentiment(headline: str, ticker: str, model_name: str = best_model_name) -> str:
    cleaned = clean_text(headline)
    text_vec = tfidf_vectorizer.transform([cleaned])
    ticker_vec = ticker_encoder.transform([[ticker.upper()]])
    X_sample = sparse.hstack([text_vec, sparse.csr_matrix(ticker_vec)])
    model = model_registry[model_name]["model"]
    return model.predict(X_sample)[0]


sample_inputs = [
    ("Apple shares jump after crushing earnings and boosting dividends", "AAPL"),
    ("Tesla faces recall and regulatory probe over autopilot crashes", "TSLA"),
    ("Microsoft trades sideways ahead of mixed macro data", "MSFT"),
]
for headline, ticker in sample_inputs:
    print(f"{ticker} | {headline}\n -> {predict_sentiment(headline, ticker)}\n")



## Step 9 – Save artifacts


In [ ]:
models_dir = MODEL_ARTIFACT_PATHS["logreg_model"].parent
models_dir.mkdir(parents=True, exist_ok=True)

joblib.dump(logreg_model, MODEL_ARTIFACT_PATHS["logreg_model"])
joblib.dump(linearsvc_model, MODEL_ARTIFACT_PATHS["linearsvc_model"])
joblib.dump(tfidf_vectorizer, MODEL_ARTIFACT_PATHS["tfidf_vectorizer"])
joblib.dump(ticker_encoder, MODEL_ARTIFACT_PATHS["ticker_encoder"])

print("Saved artifacts:")
for name, path in MODEL_ARTIFACT_PATHS.items():
    print(f" • {name}: {path}")



** Wrap up **